<a href="https://colab.research.google.com/github/ezazahamad2003/Chico_Hackathon./blob/main/sycamore_pipeline_a26c9886_296a_472b_b309_26b366009c71.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sycamore-ai[pinecone]
# DocPrep code uses the Sycamore document ETL library: https://github.com/aryn-ai/sycamore

In [ ]:
!apt-get install poppler-utils

In [ ]:
import pyarrow.fs
import sycamore
import json
import os
from pinecone import Pinecone
from sycamore.functions.tokenizer import OpenAITokenizer
from sycamore.llms import OpenAIModels, OpenAI
from sycamore.transforms import COALESCE_WHITESPACE
from sycamore.transforms.merge_elements import GreedySectionMerger
from sycamore.transforms.partition import ArynPartitioner
from sycamore.transforms.embed import OpenAIEmbedder
from sycamore.materialize_config import MaterializeSourceMode
from sycamore.utils.pdf_utils import show_pages
from sycamore.transforms.summarize_images import SummarizeImages
from sycamore.context import ExecMode
from pinecone import ServerlessSpec

In [ ]:
# API Keys
from google.colab import userdata
# Set your secrets in the colab notebook. Navigate to the left pane
# and choose the key option to set your keys. Make sure to enable Notebook access
try:
  # Visit https://www.aryn.ai/get-started to get a key.
  os.environ["ARYN_API_KEY"] = userdata.get('ARYN_API_KEY')
  os.environ["PINECONE_API_KEY"] = userdata.get('PINECONE_API_KEY')
  os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
except Exception as e:
  print("YOU ARE MISSING REQUIRED API KEYS FOR THIS PIPELINE. Add your API keys to the Secrets page (icon is a key) in the Colab left navigation panel. It is case sensitive.")

In [ ]:
# Sycamore uses lazy execution for efficiency, so the ETL pipeline will only execute when running cells with specific functions.

paths = [""C:\Users\ezaza\Downloads\Digestive_System_508-bbox.json""]

# Initialize the Sycamore context
ctx = sycamore.init(ExecMode.LOCAL)
# Set the embedding model and its parameters
model_name = "text-embedding-3-small"
max_tokens = 8191
dimensions = 1536
# Initialize the tokenizer
tokenizer = OpenAITokenizer(model_name)

ds = (
    ctx.read.binary(paths, binary_format="pdf")
    # Partition and extract tables and images
    .partition(partitioner=ArynPartitioner(
        threshold="auto",
        use_ocr=True,
        extract_table_structure=True,
        extract_images=True,
        source="docprep"
    ))
    # Use materialize to cache output. If changing upstream code or input files, change setting from USE_STORED to RECOMPUTE to create a new cache.
    .materialize(path="/content/materialize/partitioned", source_mode=MaterializeSourceMode.USE_STORED)
    # Merge elements into larger chunks
    .merge(merger=GreedySectionMerger(
      tokenizer=tokenizer,  max_tokens=max_tokens, merge_across_pages=False
    ))
    # Split elements that are too big to embed
    .split_elements(tokenizer=tokenizer, max_tokens=max_tokens)
)

ds.execute()

# Display the first 3 pages after chunking
show_pages(ds, limit=3)


In [ ]:
embedded_ds = (
    # Copy document properties to each Document's sub-elements
    ds.spread_properties(["path", "entity"])
    # Convert all Elements to Documents
    .explode()
    # Embed each Document. You can change the embedding model. Make your target vector index matches this number of dimensions.
    .embed(embedder=OpenAIEmbedder(model_name=model_name))
)
# To know more about docset transforms, please visit https://sycamore.readthedocs.io/en/latest/sycamore/transforms.html


In [ ]:
# Create an instance of ServerlessSpec with the specified cloud provider and region
spec = ServerlessSpec(cloud="aws", region="us-east-1")
index_name = "quickstart"
# Write data to a Pinecone index
embedded_ds.write.pinecone(index_name=index_name,
    dimensions=dimensions,
    distance_metric="cosine",
    index_spec=spec
)

In [ ]:
# Verify data has been loaded using DocSet Query to retrieve chunks
query_docs = ctx.read.pinecone(index_name=index_name, api_key=os.getenv('PINECONE_API_KEY'))
query_docs.show(show_embedding=False)
